In [1]:
import pandas as pd
import pandas as pd
import gzip
import pickle
import json
import os

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error
from glob import glob 

def load_data(filedata):

    dataframe = pd.read_csv(
        filedata,
        index_col=False,
        compression="zip",
    )

    return dataframe


In [2]:
################ Paso 1 ################
#Limpiar los datos
def clean_data(df):
    df_copy = df.copy()
    current_year = 2021
    columns_to_drop = ['Year', 'Car_Name']
    df_copy["Age"] = current_year - df_copy["Year"]
    df_copy = df_copy.drop(columns=columns_to_drop)
    return df_copy


In [3]:
################ Paso 2 ################
# Divida los datasets en x_train, y_train, x_test, y_test.
def split_data(df):
    #X , Y
    return df.drop(columns=["Present_Price"]), df["Present_Price"]

In [4]:
################ Paso 3 ################
# Crear un pipeline para el modelo de clasificación
def make_pipeline(x_train):
    categorical_features=['Fuel_Type','Selling_type','Transmission']
    numerical_features= [col for col in x_train.columns if col not in categorical_features]

    preprocessor = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(), categorical_features),
                ('scaler',MinMaxScaler(),numerical_features),
            ],
        )

    pipeline=Pipeline(
            [
                ("preprocessor",preprocessor),
                ('feature_selection',SelectKBest(f_regression)),
                ('classifier', LinearRegression())
            ]
        )
    return pipeline

In [6]:
################ Paso 4 ################
# Optimizar los hiperparametros del pipeline usando validación cruzada.
def create_estimator(pipeline):
    # Definición de la malla de parámetros para la búsqueda
    param_grid = {
    'feature_selection__k':range(1,25),
    'classifier__fit_intercept':[True,False],
    'classifier__positive':[True,False]

}

    # Configuración de GridSearchCV para la validación cruzada
    grid_search = GridSearchCV(
        estimator=pipeline,           # Pipeline que incluye el preprocesamiento y el clasificador
        param_grid=param_grid,        # Hiperparámetros a optimizar
        cv=10,                        # 10 divisiones para la validación cruzada
        scoring='neg_mean_absolute_error',
        n_jobs=-1,
        refit=True, 
        verbose= 1
    )

    return grid_search

In [7]:
################ Paso 5 ################
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
def _create_output_directory(output_directory):
    if os.path.exists(output_directory):
        for file in glob(f"{output_directory}/*"):
            os.remove(file)
        os.rmdir(output_directory)
    os.makedirs(output_directory)

def _save_model(path, estimator):
    _create_output_directory("files/models/")  # Verifica que el directorio se gestione correctamente.

    with gzip.open(path, "wb") as f:  # Abre el archivo comprimido en modo escritura binaria.
        pickle.dump(estimator, f)  # Guarda el modelo serializado.

In [8]:
################ Paso 6 ################
# Calcule las metricas de precision, precision balanceada, recall...
def calculate_metrics(dataset_type, y_true, y_pred):
    """Calculate metrics"""
    return {
        "type": "metrics",
        "dataset": dataset_type,
        'r2': float(r2_score(y_true, y_pred)),
        'mse': float(mean_squared_error(y_true, y_pred)),
        'mad': float(median_absolute_error(y_true, y_pred)),
    }
    

def _run_jobs():
    
    data_train = load_data("./files/input/train_data.csv.zip")
    data_test = load_data("./files/input/test_data.csv.zip")
    data_train = clean_data(data_train)
    data_test = clean_data(data_test)
    x_train, y_train = split_data(data_train)
    x_test, y_test = split_data(data_test)
    pipeline = make_pipeline(x_train)

    estimator = create_estimator(pipeline)
    estimator.fit(x_train, y_train)

    _save_model(
        os.path.join("files/models/", "model.pkl.gz"),
        estimator,
    )

    y_test_pred = estimator.predict(x_test)
    test_precision_metrics = calculate_metrics("test", y_test, y_test_pred)
    y_train_pred = estimator.predict(x_train)
    train_precision_metrics = calculate_metrics("train", y_train, y_train_pred)


    os.makedirs("files/output/", exist_ok=True)

    with open("files/output/metrics.json", "w", encoding="utf-8") as file:
        file.write(json.dumps(train_precision_metrics) + "\n")
        file.write(json.dumps(test_precision_metrics) + "\n")


In [9]:
if __name__ == "__main__":
    _run_jobs()

Fitting 10 folds for each of 96 candidates, totalling 960 fits
